# Importing Relevant Libraries

In [ ]:
import os

import numpy as np
import tensorflow as tf

In [1]:
labels = []
data = []

# Little data preprocessing 

## Accessing dataset that has been collected in previous notebook

In [2]:
for label in range(3):
    class_dir = os.path.join(save_dir, 'class_{}'.format(label))
    print(class_dir)
    for file_name in os.listdir(class_dir):
        file_path = os.path.join(class_dir, file_name)
        landmark = np.load(file_path)
        labels.append(label)
        data.append(landmark)

NameError: name 'os' is not defined

In [ ]:
# Define the hand gesture labels
labels = ['Rock', 'Paper', 'Scissors']
#landmarks = []
# Initialize the MediaPipe Hand module
mp_hands_2 = mp.solutions.hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

while True:
    # Read frames from the webcam
    ret, frame = cap.read()

    # Convert the frame to RGB for input to MediaPipe
    #frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    if not ret:
        break
    
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False 
    results = mp_hands_2.process(image)

    image.flags.writeable = True 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Detect hand landmarks using MediaPipe
    #results = mp_hands_2.process(frame_rgb)

    # Check if hand landmarks are detected
     
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks
                                      , mp_hands.HAND_CONNECTIONS)
        # Extract the landmark coordinates
            image_height, image_width, _ = image.shape

            landmark_list = []

            for landmark in hand_landmarks.landmark: 
                landmark_x = landmark.x * image_width
                landmark_y = landmark.y * image_height

                landmark_list.extend([landmark_x, landmark_y, landmark.z])
            #landmarks.append(landmark_list)
            
            landmark_array = np.array(landmark_list)
            landmark_array = np.expand_dims(landmark_array, axis=0)
            # Perform prediction
            print(len(landmark_list))
            prediction = model.predict(landmark_array)
            predicted_class = np.argmax(prediction)
            gesture_label = labels[predicted_class]

            # Draw the predicted gesture label on the frame
            cv2.putText(frame, gesture_label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the frame
    cv2.imshow('Hand Gesture Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
dataset = list(zip(data, labels))
random.shuffle(dataset)
data_, labels_ = zip(*dataset)


In [ ]:
dataset[0][0]


In [ ]:
data = np.array(data_)
labels = np.array(labels_)


In [ ]:
train_landmarks = data[:-200]
train_labels = labels[:-200]
test_landmarks = data[-200:]
test_labels = labels[-200:]


In [ ]:
num_landmarks = 21  # Number of hand landmarks detected by MediaPipe
num_classes = 3  # Number of hand gesture classes (rock, paper, scissors)

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu',
                          input_shape=(num_landmarks * 3,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_landmarks, train_labels, epochs=15)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_landmarks, test_labels)
print('Test accuracy:', test_acc)


In [ ]:
model.save(".")


In [ ]:
cap = cv2.VideoCapture(0)


In [ ]:
cap.release()
cv2.destroyAllWindows()
